###### Reference: 
    https://finthon.com/learn-cnn-two-tfrecord-read-data/
    https://finthon.com/learn-cnn-three-resnet-prediction/

# 匯入圖片資料並輸出成tfrecord檔案

In [4]:
import os
from PIL import Image
import tensorflow as tf

In [5]:
'''
設置路徑
# 將需分類之圖片目錄放置Working Directory於之下，Folder以Int作為命名
'''
# 图片路径，两组标签都在该目录下
cwd = r"./OM/"
# tfrecord文件保存路径
file_path = r"./"
# 每个tfrecord存放图片个数
bestnum = 10000
# 第几个图片
num = 0
# 第几个TFRecord文件
recordfilenum = 0
# 将labels放入到classes中
classes = []
for i in os.listdir(cwd):
    classes.append(i)
# tfrecords格式文件名
ftrecordfilename = ("traindata_63.tfrecords-%.3d" % recordfilenum)
writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))

In [6]:
'''
tfrecord 製造函數
'''
for index, name in enumerate(classes):
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num = num + 1
        if num > bestnum:   
            num = 1
            recordfilenum += 1
            ftrecordfilename = ("traindata_63.tfrecords-%.3d" % recordfilenum)
            writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))
        
        img_path = os.path.join(class_path, img_name)  # 每一个图片的地址
        img = Image.open(img_path, 'r')
        img_raw = img.tobytes()  # 将图片转化为二进制格式
        example = tf.train.Example(
            features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            }))
        writer.write(example.SerializeToString())  # 序列化为字符串
writer.close()

# 自tfrecord輸入資料集

In [ ]:
'''
tfrecord 輸入函數
'''
import tensorflow as tf
 
def read_and_decode_tfrecord(filename):
    filename_deque = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_deque)
    features = tf.parse_single_example(serialized_example, features={
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string)})
    label = tf.cast(features['label'], tf.int32)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [640, 480, 3])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

# 訓練CNN模型

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

In [ ]:
'''
定义模型保存地址，batch_sizes设置的小一点训练效果更好，将当前目录下的tfrecord文件放入列表中:
# tf.train.shuffle_batch: 随机打乱队列里面的数据顺序
# num_threads: 表示线程数
# capacity" 表示队列的容量，在这里设置成10000
# min_after_dequeue: 队列里保留的最小数据量，并且控制着随机的程度，
设置成9900的意思是，当队列中的数据出列100个，剩下9900个的时候，就要重新补充100个数据进来并打乱顺序。
如果你要按顺序导入队列，改成tf.train.batch函数，并删除min_after_dequeue参数。
这些参数都要根据自己的电脑配置进行相应的设置。
接下来将label值进行onehot编码，直接调用tf.one_hot函数。因为我们这里有2类，depth设置成2:
'''

save_dir = r"./train_image_63.model"  # 模型保存路径
batch_size_ = 2
lr = tf.Variable(0.0001, dtype=tf.float32)  # 学习速率
x = tf.placeholder(tf.float32, [None, 640, 480, 3])  # 图片大小为224*224*3
y_ = tf.placeholder(tf.float32, [None])

'''
train_list = ['traindata_63.tfrecords-000', 'traindata_63.tfrecords-001', 'traindata_63.tfrecords-002',
              'traindata_63.tfrecords-003', 'traindata_63.tfrecords-004', 'traindata_63.tfrecords-005',
              'traindata_63.tfrecords-006', 'traindata_63.tfrecords-007', 'traindata_63.tfrecords-008',
              'traindata_63.tfrecords-009', 'traindata_63.tfrecords-010', 'traindata_63.tfrecords-011',
              'traindata_63.tfrecords-012', 'traindata_63.tfrecords-013', 'traindata_63.tfrecords-014',
              'traindata_63.tfrecords-015', 'traindata_63.tfrecords-016', 'traindata_63.tfrecords-017',
              'traindata_63.tfrecords-018', 'traindata_63.tfrecords-019', 'traindata_63.tfrecords-020',
              'traindata_63.tfrecords-021']    #制作成的所有tfrecord数据，每个最多包含1000个图片数据
'''

train_list = ['traindata_63.tfrecords-000']

# 随机打乱顺序
img, label = read_and_decode_tfrecord(train_list)
img_batch, label_batch = tf.train.shuffle_batch([img, label], num_threads=2, batch_size=batch_size_, capacity=10000,
                                                min_after_dequeue=9900)

In [ ]:
'''
接下来将label值进行onehot编码，直接调用tf.one_hot函数。因为我们这里有100类，depth设置成100:
'''
# 将label值进行onehot编码
one_hot_labels = tf.one_hot(indices=tf.cast(y_, tf.int32), depth=2)
pred, end_points = nets.resnet_v2.resnet_v2_50(x, num_classes=2, is_training=True)
pred = tf.reshape(pred, shape=[-1, 2])

In [ ]:
'''
# nets.resnet_v2.resnet_v2_50: 直接调用ResNet_50网络
# num_classes等于类别总数
# is_training表示我们是否要训练网络里面固定层的参数，True表示所有参数都重新训练，False表示只训练后面几层的参数。
网络搭好后，我们继续定义损失函数和优化器，损失函数选择sigmoid交叉熵，优化器选择Adam：
'''
# 定义损失函数和优化器
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=one_hot_labels))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [ ]:
# 准确度
a = tf.argmax(pred, 1)
b = tf.argmax(one_hot_labels, 1)
correct_pred = tf.equal(a, b)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 创建一个协调器，管理线程
    coord = tf.train.Coordinator()
    # 启动QueueRunner,此时文件名队列已经进队
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    i = 0
    while True:
        i += 1
        b_image, b_label = sess.run([img_batch, label_batch])
        _, loss_, y_t, y_p, a_, b_ = sess.run([optimizer, loss, one_hot_labels, pred, a, b], feed_dict={x: b_image,
                                                                                                        y_: b_label})
        print('step: {}, train_loss: {}'.format(i, loss_))
        if i % 20 == 0:
            _loss, acc_train = sess.run([loss, accuracy], feed_dict={x: b_image, y_: b_label})
            print('--------------------------------------------------------')
            print('step: {}  train_acc: {}  loss: {}'.format(i, acc_train, _loss))
            print('--------------------------------------------------------')
            if i == 200:
                saver.save(sess, save_dir, global_step=i)
            #elif i == 300000:
            #    saver.save(sess, save_dir, global_step=i)
            #elif i == 400000:
            #    saver.save(sess, save_dir, global_step=i)
                break
    coord.request_stop()
    # 其他所有线程关闭之后，这一函数才能返回
    coord.join(threads)

# 使用模型進行預測

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets
from PIL import Image
import os
 
test_dir = r'./test'     # 原始的test文件夹，含带预测的图片
model_dir = r'./train_image_63.model-300000'     # 模型地址
test_txt_dir = r'./test.txt'     # 原始的test.txt文件
result_dir = r'./result.txt'     # 生成输出结果
x = tf.placeholder(tf.float32, [None, 640, 480, 3])

'''
classes = ['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24',
           '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40',
           '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57',
           '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73',
           '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9',
           '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']      # 标签顺序
'''

classes = ['0', '1']      # 标签顺序

pred, end_points = nets.resnet_v2.resnet_v2_50(x, num_classes=2, is_training=True)
pred = tf.reshape(pred, shape=[-1, 2])
a = tf.argmax(pred, 1)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, model_dir)
    with open(test_txt_dir, 'r') as f:
        data = f.readlines()
        for i in data:
            test_name = i.split()[0]
            for pic in os.listdir(test_dir):
                if pic == test_name:
                    img_path = os.path.join(test_dir, pic)
                    img = Image.open(img_path)
                    img = img.resize((640, 480))
                    img = tf.reshape(img, [1, 640, 480, 3])
                    img1 = tf.reshape(img, [1, 640, 480, 3])
                    img = tf.cast(img, tf.float32) / 255.0
                    b_image, b_image_raw = sess.run([img, img1])
                    t_label = sess.run(a, feed_dict={x: b_image})
                    index_ = t_label[0]
                    predict = classes[index_]
                    with open(result_dir, 'a') as f1:
                        print(test_name, predict, file=f1)
                    break